In [12]:
import numpy as np
import math
import time
import pandas as pd
from tqdm import tqdm
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline

import data
import features
from model_tree_regressor import ModelTreeRegressor

In [13]:
temp, dico = data.load_input_data('data/train_input.csv')
output = data.load_output_data('data/challenge_output.csv', temp, dico)
all_features = features.load_all_features(dico, temp, remove_useless=True)

100%|████████████████████████████████████████████████████████████████████████████████| 240/240 [00:12<00:00, 18.69it/s]


All the features have been loaded in 12.85 sec
16 useless features have been removed. There are now 29 features for each setting.


In [15]:
def test_type_feature(t):
    print('-- {} --'.format(t))
    name_features = features.choose_name_features(all_features, t)
    model = ModelTreeRegressor(all_features, output)

    # Metrics sur toutes les variables à prédire. Moyenne sur N separations train/test
    results = model.cross_validate(features_names=name_features, N=5, pct_train=0.75, do_print=True)

### 1. non_int

In [16]:
test_type_feature('_non_int')


-- _non_int --
Evolution of the 5 iterations:


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:21<00:00, 40.22s/it]


Average mse of office_temperature_degreC: 0.8235704548005647
Average mse / max of office_temperature_degreC: 0.0009622726569008885
Average mse of Q_total_heating_kW: 811.2005273292793
Average mse / max of Q_total_heating_kW: 0.9466795895290208
Average mse of Q_total_AC_kW: 619.0877437330583
Average mse / max of Q_total_AC_kW: 0.7078179045354595
Average mse of Q_total_gains_kW: 47.65515111419735
Average mse / max of Q_total_gains_kW: 0.05492848225499032
Average mse of Q_total_kW: 1494.5653386240733
Average mse / max of Q_total_kW: 1.730616004667939


In [17]:
test_type_feature('_without_lever')


-- _without_lever --
Evolution of the 5 iterations:


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:37<00:00, 43.58s/it]


Average mse of office_temperature_degreC: 1.5796066620905558
Average mse / max of office_temperature_degreC: 0.001819763868942228
Average mse of Q_total_heating_kW: 1333.111881108479
Average mse / max of Q_total_heating_kW: 1.5263570282258123
Average mse of Q_total_AC_kW: 898.8089011286013
Average mse / max of Q_total_AC_kW: 1.000682910741181
Average mse of Q_total_gains_kW: 312.62008149837
Average mse / max of Q_total_gains_kW: 0.3622289548805259
Average mse of Q_total_kW: 2400.422842813271
Average mse / max of Q_total_kW: 2.6912859102979767


In [18]:
test_type_feature('_without_lever_non_int')


-- _without_lever_non_int --
Evolution of the 5 iterations:


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:09<00:00, 37.90s/it]


Average mse of office_temperature_degreC: 1.2033461200187685
Average mse / max of office_temperature_degreC: 0.0013367452374510097
Average mse of Q_total_heating_kW: 863.0815190679648
Average mse / max of Q_total_heating_kW: 0.961850613159631
Average mse of Q_total_AC_kW: 539.5324488083763
Average mse / max of Q_total_AC_kW: 0.5888223022519746
Average mse of Q_total_gains_kW: 41.44547554892913
Average mse / max of Q_total_gains_kW: 0.04229267426196157
Average mse of Q_total_kW: 1591.2419794714128
Average mse / max of Q_total_kW: 1.7570068535361307


In [20]:
test_type_feature('')


--  --
Evolution of the 5 iterations:


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:31<00:00, 42.36s/it]


Average mse of office_temperature_degreC: 0.9789263009453253
Average mse / max of office_temperature_degreC: 0.0011142487422904563
Average mse of Q_total_heating_kW: 1121.375446854963
Average mse / max of Q_total_heating_kW: 1.2754848307694076
Average mse of Q_total_AC_kW: 805.0903191724259
Average mse / max of Q_total_AC_kW: 0.8912393926363331
Average mse of Q_total_gains_kW: 43.069662443024164
Average mse / max of Q_total_gains_kW: 0.04560151580818748
Average mse of Q_total_kW: 2084.773274807406
Average mse / max of Q_total_kW: 2.3312420972610015


|        | office temperature | total heating | total_AC | total_gains | total |
|------- |--------------------|---------------|----------|-------------|-------|
| classique |  1.09           | 1167.14       |  849.87  | 8.007       |1888.44|
| non_int|  0.82              |  811.20       |  619.09  |  47.66      |1494.57|
| without lever|  1.58        | 1333.11       |  898.81  |  312.62     |2400.42|
| without lever AND non int| 1.20 | 863.08    |  539.53  |  41.44      |1591.24|

In [21]:
name_features = features.choose_name_features(all_features, '_non_int')

models = {}
# Train model on all the data
model.load_train_test_set(name_features, pct_train=1.)

for var in model.output_names:
    print("Learning on '{}'...".format(var))
    model.fit_model(var)
    models[var] = model.model

Learning on 'office_temperature_degreC'...
Learning on 'Q_total_heating_kW'...
Learning on 'Q_total_AC_kW'...
Learning on 'Q_total_gains_kW'...
Learning on 'Q_total_kW'...


In [22]:
X_val = features.load_data_features('./data/test_input.csv', name_features, remove_useless=True)
model.save_output(models, './data/test_tree_non_int.csv', X_val)

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:01<00:00, 19.26it/s]


All the features have been loaded in 1.88 sec
16 useless features have been removed. There are now 29 features for each setting.


100%|███████████████████████████████████████████████████████████████████████| 314496/314496 [00:15<00:00, 20336.72it/s]


File saved at './data/test_tree_non_int.csv'
